<h1 id="課題---Human-Resources">課題 - Human Resources<a class="anchor-link" href="#課題---Human-Resources">¶</a></h1>

<h3>以下は過去4か月で学習したものをベースに課題を解いた流れをまとめてみました。<br>
個人のメモとして残しておく目的もありますので、ご了承ください。…櫻井</h3>

１．
まずファイルを読み込む。<br>
left 列が目的変数なので最右に配置。<br>


In [4]:
import pandas as pd

# データの読み込み
df = pd.read_csv('./data/final_hr_analysis_train.csv', header=0)
X = df.iloc[:, 2:]
y = df.iloc[:, 1]

# 2値分類の対象left列を最右に再配置
display(X.join(y).head())
X.join(y).dtypes

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales,salary,left
0,0.53,0.52,2,135,4,0,0,technical,medium,0
1,0.77,0.53,5,256,3,0,0,accounting,medium,0
2,0.89,0.79,3,149,2,0,0,support,medium,1
3,0.64,0.63,3,156,6,1,0,support,low,0
4,0.98,0.74,4,151,3,0,0,sales,medium,0


satisfaction_level       float64
last_evaluation          float64
number_project             int64
average_montly_hours       int64
time_spend_company         int64
Work_accident              int64
promotion_last_5years      int64
sales                     object
salary                    object
left                       int64
dtype: object

２．データを見ると欠損値はない。<br>
そこで前処理はOne hot encodingのみ。

In [5]:
#OneHotエンコーディング
ohe_columns = ['sales','salary']
X_ohe= pd.get_dummies(X, dummy_na=False, columns=ohe_columns)
display(X_ohe.head())
print('X_ohe shape: (%i,%i)' %X_ohe.shape)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.53,0.52,2,135,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,0.77,0.53,5,256,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1
2,0.89,0.79,3,149,2,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
3,0.64,0.63,3,156,6,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0
4,0.98,0.74,4,151,3,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1


X_ohe shape: (10499,20)


３．交叉検証のためSplit。<br>
KNN、ロジスティック回帰、ランダムフォレスト,勾配ブースティング,多層パーセプトロン,線形SVC　の6種類でPipeLine

In [9]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# 交叉検証 holdout
X_train,X_test,y_train,y_test = train_test_split(X_ohe, y, test_size=0.20, random_state=1)

# Pipelines ：KNN、ロジスティック回帰、ランダムフォレスト,勾配ブースティング,多層パーセプトロン,線形SVC
pipe_knn = Pipeline([('scl',StandardScaler()), ('est',KNeighborsClassifier())])
pipe_log = Pipeline([('scl', StandardScaler()), ('est',LogisticRegression(random_state=1))])
pipe_rft = Pipeline([('scl',StandardScaler()),('est',RandomForestClassifier(random_state=1))])
pipe_gbt = Pipeline([('scl',StandardScaler()),('est',GradientBoostingClassifier(random_state=1))])
pipe_mlp = Pipeline([('scl',StandardScaler()),('est',MLPClassifier(hidden_layer_sizes=(5,2), max_iter=500, random_state=1))])
pipe_lsv = Pipeline([('scl', StandardScaler()), ('est', LinearSVC())])

pipe_names = ['KNN','Logistic','RandomForest','GradientBoosting','MLP','LinearSVC']
pipe_lines = [pipe_knn, pipe_log, pipe_rft, pipe_gbt, pipe_mlp, pipe_lsv]

４． パイプラインを学習。<br>
F値だけでなく、適合率、再現率、AUCも出力。<br>
各PipeLineをスコア種類別に配列にして、sortして出力することでランキングがわかるように。 

In [11]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from operator import itemgetter, attrgetter

list_pre =[]
list_rec =[]
list_f1 =[]
list_auc =[]

#u=pipe_rft.predict_proba(X_ohe)
#print(u[0:50])

# パイプラインの学習・評価
for (i,pipe) in enumerate(pipe_lines):
    
    pipe.fit(X_train, y_train.as_matrix().ravel())
    y_pred=pipe.predict(X_test)
    
    list_pre.append([pipe_names[i], precision_score(y_test.as_matrix().ravel(), y_pred)])
    list_rec.append([pipe_names[i], recall_score(y_test.as_matrix().ravel(), y_pred)])
    list_f1.append([pipe_names[i], f1_score(y_test.as_matrix().ravel(), y_pred)])
    list_auc.append([pipe_names[i], roc_auc_score(y_test.as_matrix().ravel(), y_pred)])

#ベストスコアをランキング順    
print('\n***適合率順********')                    
list_pre=sorted(list_pre, key=itemgetter(1), reverse=True)
for i,x in enumerate(list_pre):
    print ('%s. %.3f: %s '%(i+1, x[1], x[0]))
                    
print('\n***再現率順********')
list_rec=sorted(list_rec, key=itemgetter(1), reverse=True)
for i,x in enumerate(list_rec):
    print ('%s. %.3f: %s '%(i+1, x[1], x[0]))
    
print('\n***F値順 ****************')
list_f1=sorted(list_f1, key=itemgetter(1), reverse=True)
for i,x in enumerate(list_f1):
    print ('%s. %.3f: %s '%(i+1, x[1], x[0]))        
    
print('\n***AUC順 ****************')
list_auc=sorted(list_auc, key=itemgetter(1), reverse=True)
for i,x in enumerate(list_auc):
    print ('%s. %.3f: %s '%(i+1, x[1], x[0]))     


***適合率順********
1. 0.992: RandomForest 
2. 0.962: GradientBoosting 
3. 0.922: MLP 
4. 0.874: KNN 
5. 0.614: Logistic 
6. 0.574: LinearSVC 

***再現率順********
1. 0.938: RandomForest 
2. 0.924: GradientBoosting 
3. 0.903: MLP 
4. 0.874: KNN 
5. 0.366: Logistic 
6. 0.296: LinearSVC 

***F値順 ****************
1. 0.964: RandomForest 
2. 0.942: GradientBoosting 
3. 0.912: MLP 
4. 0.874: KNN 
5. 0.459: Logistic 
6. 0.390: LinearSVC 

***AUC順 ****************
1. 0.968: RandomForest 
2. 0.956: GradientBoosting 
3. 0.939: MLP 
4. 0.916: KNN 
5. 0.646: Logistic 
6. 0.612: LinearSVC 


RandomForestがベスト。<br> 
しかし勾配ブースティング,多層パーセプトロンも悪くない。<br> <br> <br> 

５．３つのモデルをPKLに保存しておく。


In [20]:
from sklearn.externals import joblib
#RandomForest が No1 と判定, PKL形式でモデル保存
model_rft=pipe_rft.fit(X_train, y_train.as_matrix().ravel())
file_rft='finalRFT_0422_sakurai.pkl'
joblib.dump(model_rft, file_rft, compress=3) 
print(file_rft+"：保存")

#勾配ブースティング が No2と判定, PKL形式でモデル保存
model_gbt=pipe_gbt.fit(X_train, y_train.as_matrix().ravel())
file_gbt='finalGBT_0422_sakurai.pkl'
joblib.dump(model_gbt, file_gbt, compress=3)   
print(file_gbt+"：保存")

#多層パーセプトロンが　No3と判定, PKL形式でモデル保存
model_mlp=pipe_mlp.fit(X_train, y_train.as_matrix().ravel())
file_mlp='finalMLP_0422_sakurai.pkl'
joblib.dump(model_mlp, file_mlp, compress=3)
print(file_mlp+"：保存")

finalRFT_0422_sakurai.pkl：保存
finalGBT_0422_sakurai.pkl：保存
finalMLP_0422_sakurai.pkl：保存


６．検証用のファイルを読み込む。→　同じようにOneHotエンコーディング。<br>


In [24]:
#検証用のファイルを読み込み
df2 = pd.read_csv('./data/final_hr_analysis_test.csv', header=0)
X2 = df2.iloc[:, 2:]
y2 = df2.iloc[:, 1]

#OneHotエンコーディング
ohe_columns = ['sales','salary']
X_ohe2= pd.get_dummies(X2, dummy_na=False, columns=ohe_columns)
display(X_ohe2.head())
print('X_ohe shape: (%i,%i)' %X_ohe2.shape)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,promotion_last_5years,sales_IT,sales_RandD,sales_accounting,sales_hr,sales_management,sales_marketing,sales_product_mng,sales_sales,sales_support,sales_technical,salary_high,salary_low,salary_medium
0,0.44,0.57,2,141,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
1,0.55,0.96,3,194,3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
2,0.72,0.67,5,210,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
3,0.96,0.75,4,177,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
4,0.96,0.54,3,198,3,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


X_ohe shape: (4500,20)


７．３つのモデルの予測確率をpredict_probaで出力してみる。（とりあえず先頭の10行のみ表示）

In [29]:
from sklearn.externals import joblib
model_rft = joblib.load(file_rft)
model_gbt = joblib.load(file_gbt)
model_mlp = joblib.load(file_mlp)

#pred2=loadModel.predict(X_ohe2)
from decimal import Decimal, ROUND_HALF_UP

#クラス=1に該当する確率を各IDごとに求める　predict_proba
print('\n***ランダムフォレスト (確率)****************')
z1=model_rft.predict_proba(X_ohe2)[:,] 
print(z1[0:10])

print('\n***勾配ブースティング (確率)****************')
z2=model_gbt.predict_proba(X_ohe2)[:,]
print(z2[0:10])

print('\n***多層パーセプトロン (確率)****************')
z3=model_mlp.predict_proba(X_ohe2)[:,]
print(z3[0:10])


***ランダムフォレスト (確率)****************
[[ 0.  1.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 [ 0.  1.]]

***勾配ブースティング (確率)****************
[[ 0.05515594  0.94484406]
 [ 0.98868322  0.01131678]
 [ 0.98866254  0.01133746]
 [ 0.99737923  0.00262077]
 [ 0.99682379  0.00317621]
 [ 0.98708724  0.01291276]
 [ 0.88831933  0.11168067]
 [ 0.99001358  0.00998642]
 [ 0.98838473  0.01161527]
 [ 0.03190534  0.96809466]]

***多層パーセプトロン (確率)****************
[[ 0.23054328  0.76945672]
 [ 0.98318376  0.01681624]
 [ 0.98318376  0.01681624]
 [ 0.98318376  0.01681624]
 [ 0.98318376  0.01681624]
 [ 0.98318376  0.01681624]
 [ 0.84576672  0.15423328]
 [ 0.98318376  0.01681624]
 [ 0.98318376  0.01681624]
 [ 0.02959921  0.97040079]]


ランダムフォレストはベストスコアではあるが、予測確率としては大雑把過ぎる。<br>
一方、勾配ブースティング、多層パーセプトロンのような値が出せたら客に納得してもらいやすい。<br>
全部利用してより精緻な確率を出してはどうか。<br><br>

８．そこで各確率をF値でかけておき、３モデルの平均を出すことを思いつく。<br>
各モデルの良さを生かし、より精緻になるかもしれない。<br>
ネットを探してみたがそのような事例やサンプルは見つからず。<br>
邪道なのかも？　でもまあやってみる。

In [33]:
#予測値にF値をかけておく（F値は直書き）
z1=model_rft.predict_proba(X_ohe2)[:,-1] * 0.964
z2=model_gbt.predict_proba(X_ohe2)[:,-1] * 0.942
z3=model_mlp.predict_proba(X_ohe2)[:,-1] * 0.912

ids = df2.iloc[:, 0]
fileName2='FINALPredProba2_aijcn1014.csv'

import csv
f=open(fileName2,'w')
writer=csv.writer(f,lineterminator='\n')

csvlist=[]
csvlist.append('index')
csvlist.append('1の確率') 
writer.writerow(csvlist)

j=[0,0,0]
tempVal=0

for i in range(0,ids.size):
    csvlist=[]
    
    #1列目：IDを出力
    csvlist.append(ids[i])
    #2列目：３つのモデルの平均値を出力　
    csvlist.append((z1[i]+z2[i]+z3[i])/3)
    
    i=i+1
    writer.writerow(csvlist)

f.close()
print (fileName2+': 出力数 %s: ' %(i))

FINALPredProba2_aijcn1014.csv: 出力数 4500: 


CSVファイルを開くとそれなりに確率が出力されている。<br>
ランダムフォレストのみの予測値よりは、人が納得できそう。<br><br>

それ以外に方法も思いつかず、ひとまずこれを提出用とする。